# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [8]:
KEYWORDS = ['Тополиный', 'технологии']

In [3]:
req = requests.get('https://habr.com/ru/all/')
req.text

'<!DOCTYPE html>\n<html lang="ru" class="no-js">\n  <head>\n    <meta http-equiv="content-type" content="text/html; charset=utf-8" />\n<meta content=\'width=1024\' name=\'viewport\'>\n<title>Все публикации подряд / Хабр</title>\n\n\n\n  <meta property="fb:app_id" content="444736788986613" />\n<meta property="og:type" content="website"/>\n<meta property="fb:pages" content="472597926099084"/>\n<meta property="og:site_name" content="Хабр" />\n<link rel="image_src" href="https://habr.com/images/habr_ru.png" />\n<meta property="og:image" content="https://habr.com/images/habr_ru.png" />\n<meta property="og:image:width" content="1200" />\n<meta property="og:image:height" content="628" />\n<meta property="og:title" content="Все публикации подряд / Хабр"/>\n<meta property="og:description" content="Хабр — крупнейший в Европе ресурс для IT-специалистов. Сюда приходят обсудить новости индустрии и поделиться опытом."/>\n\n\n\n<meta name=\'yandex-verification\' content=\'71593b225aeafc4e\' />\n<meta

In [4]:
soup = BeautifulSoup(req.text, 'html.parser')
soup

<!DOCTYPE html>

<html class="no-js" lang="ru">
<head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="width=1024" name="viewport"/>
<title>Все публикации подряд / Хабр</title>
<meta content="444736788986613" property="fb:app_id">
<meta content="website" property="og:type"/>
<meta content="472597926099084" property="fb:pages"/>
<meta content="Хабр" property="og:site_name"/>
<link href="https://habr.com/images/habr_ru.png" rel="image_src"/>
<meta content="https://habr.com/images/habr_ru.png" property="og:image"/>
<meta content="1200" property="og:image:width"/>
<meta content="628" property="og:image:height"/>
<meta content="Все публикации подряд / Хабр" property="og:title"/>
<meta content="Хабр — крупнейший в Европе ресурс для IT-специалистов. Сюда приходят обсудить новости индустрии и поделиться опытом." property="og:description"/>
<meta content="71593b225aeafc4e" name="yandex-verification"/>
<meta content="unsafe-url" name="referrer"/>
<meta content

In [7]:
# извлекаем посты
posts = soup.find_all('article', class_='post')
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/Boomburum/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/353/8eb/088/3538eb088997a0fd418ce85841dd0996.png" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">Boomburum</span>
 </a>
 <span class="post__time">сегодня в 16:20</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/habr/blog/513314/">Тополиный пух, AMA #20, июль</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/company/habr/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Блог компании Хабр'); }" rel="nofollow" title="Вы не подписаны на этот хаб">Блог комп

In [9]:
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    print('post', post_id)
    hubs = post.find_all('a', class_='hub-link')

post 513314
post 513310
post 513312
post 513290
post 512678
post 513306
post 513138
post 513304
post 513302
post 513300
post 513292
post 513086
post 513298
post 512350
post 513296
post 513046
post 513282
post 513288
post 513284
post 513278


In [10]:
# добавляем извлечение хабов из постов, чтобы отбирать только нужные
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('a', class_='hub-link')
    for hub in hubs:
            hub_lower = hub.text.lower()
           # ищем вхождение хотя бы одного желаемого хаба
            if any([hub_lower in desired for desired in KEYWORDS]):
               # пост нам интересен - делаем с ним все что захотим:
               # можно отправит в телеграм уведомление, можно на почту и т.п.
                title_element = post.find('a', class_='post__title_link')
                print(title_element.text, title_element.attrs.get('href'))
               # так как пост уже нам подошел - дальше нет смысла проверять хабы
                break

In [11]:
posts = soup.find_all('article', class_='post')
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/Boomburum/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/353/8eb/088/3538eb088997a0fd418ce85841dd0996.png" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">Boomburum</span>
 </a>
 <span class="post__time">сегодня в 16:20</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/company/habr/blog/513314/">Тополиный пух, AMA #20, июль</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/company/habr/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Блог компании Хабр'); }" rel="nofollow" title="Вы не подписаны на этот хаб">Блог комп

In [13]:
# добираемся до текста со ссылкой
articles_intro = list(map(lambda x: x.find('a', class_='post__title_link'), posts))
articles_intro

[<a class="post__title_link" href="https://habr.com/ru/company/habr/blog/513314/">Тополиный пух, AMA #20, июль</a>,
 <a class="post__title_link" href="https://habr.com/ru/company/selectel/blog/513310/">Bitdefender открыла код технологии интроспекции гипервизора HVI</a>,
 <a class="post__title_link" href="https://habr.com/ru/post/513312/">Учет рабочего времени с расчетом баланса</a>,
 <a class="post__title_link" href="https://habr.com/ru/company/mailru/blog/513290/">Как реализованы JIT-компиляторы</a>,
 <a class="post__title_link" href="https://habr.com/ru/company/neobit/blog/512678/">1000 и 1 способ обойти Safe Exam Browser</a>,
 <a class="post__title_link" href="https://habr.com/ru/company/timeweb/blog/513306/">12 идей для разработки проектов, которыми точно будут пользоваться люди</a>,
 <a class="post__title_link" href="https://habr.com/ru/post/513138/">Прием всего Bluetooth разом на SDR с CUDA? Легко</a>,
 <a class="post__title_link" href="https://habr.com/ru/company/JetBrains/blog/

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)```
GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
